In [1]:
%load_ext autotime

import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from heapq import nlargest
import warnings
import numpy as np
warnings.filterwarnings('ignore')

pd.set_option("display.max_colwidth",200)
pd.set_option("display.max_columns",20)
pd.set_option('float_format', '{:.3f}'.format)

time: 609 ms


In [2]:
# path = "C://Users//natha//Desktop//Capstone_local_save//Orginal Dataset"
cleaned_df = pd.read_csv("C://Users//natha//OneDrive - The University of Sydney (Students)//CS48-CAPSTONE Project 2021 Sem1//dataset//Cleaned Dataset//rm_oscillated_cleaned_data.txt", delimiter=',', \
    names=['uid','datetime','loc','app_id','traffic'], \
    dtype={'uid': str, 'datetime': str, 'loc': str, 'app_id': str, 'trafific': float})

cleaned_df['datetime'] = cleaned_df['datetime'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S')) # Convert to datetime object
cleaned_df['day'] = cleaned_df['datetime'].dt.strftime("%m-%d")

n_users = len(cleaned_df['uid'].unique())
print("Successfully loaded data")
print("-"*30)
print('Number of users', n_users)
print(cleaned_df.head())

Successfully loaded data
------------------------------
Number of users 697
  uid            datetime   loc app_id  traffic    day
0   0 2016-04-21 08:42:21  8194    342    0.030  04-21
1   0 2016-04-21 08:44:12  8194      1    0.008  04-21
2   0 2016-04-21 08:44:47  8194    857    0.027  04-21
3   0 2016-04-21 08:44:48  8194    857    0.002  04-21
4   0 2016-04-21 08:44:49  8194     31    0.009  04-21
time: 56.2 s


In [3]:
# Load App2Category data
with open('C:\\Users\\natha\\Desktop\\Capstone_local_save\\Orginal Dataset\\App2Category.txt') as f:

    lines = f.readlines()
    df = []
    for line in lines:
        data = line.split()
        df.append(data)
app2cat = pd.DataFrame(df, columns=['app_id','cat_id'])
app2cat.head()

,app_id,cat_id
0,1,4
1,2,4
2,3,4
3,4,13
4,5,4


time: 16 ms


In [4]:
cleaned_df_cat = pd.merge(cleaned_df, app2cat, on="app_id")
cleaned_df_cat.head()

,uid,datetime,loc,app_id,traffic,day,cat_id
0,0,2016-04-21 08:42:21,8194,342,0.030,04-21,0
1,0,2016-04-21 11:06:36,7210,342,0.004,04-21,0
2,0,2016-04-21 11:06:39,7210,342,0.002,04-21,0
3,0,2016-04-21 18:44:24,7549,342,0.001,04-21,0
4,0,2016-04-21 18:44:28,7549,342,0.002,04-21,0


time: 1.06 s


# Testing on user 0

In [6]:
user_0_data = cleaned_df_cat[cleaned_df_cat["uid"] == str(0)]
user_0_data = user_0_data.sort_values(by = "datetime")
user_0_data

,uid,datetime,loc,app_id,traffic,day,cat_id
0,0,2016-04-21 08:42:21,8194,342,0.030,04-21,0
1636,0,2016-04-21 08:44:12,8194,1,0.008,04-21,4
345626,0,2016-04-21 08:44:47,8194,857,0.027,04-21,8
345627,0,2016-04-21 08:44:48,8194,857,0.002,04-21,8
472513,0,2016-04-21 08:44:49,8194,612,0.001,04-21,8
390870,0,2016-04-21 08:44:49,8194,4,0.002,04-21,13
347033,0,2016-04-21 08:44:49,8194,31,0.009,04-21,8
471142,0,2016-04-21 08:44:49,8194,529,0.002,04-21,8
345628,0,2016-04-21 08:44:49,8194,857,0.579,04-21,8
347034,0,2016-04-21 08:44:50,8194,31,0.307,04-21,8


time: 485 ms


In [7]:
user_0_data_first3days = user_0_data[(user_0_data['day'] == '04-21') |(user_0_data['day'] == '04-22')| (user_0_data['day'] == '04-25')]
user_0_data_first3days

,uid,datetime,loc,app_id,traffic,day,cat_id
0,0,2016-04-21 08:42:21,8194,342,0.030,04-21,0
1636,0,2016-04-21 08:44:12,8194,1,0.008,04-21,4
345626,0,2016-04-21 08:44:47,8194,857,0.027,04-21,8
345627,0,2016-04-21 08:44:48,8194,857,0.002,04-21,8
472513,0,2016-04-21 08:44:49,8194,612,0.001,04-21,8
390870,0,2016-04-21 08:44:49,8194,4,0.002,04-21,13
347033,0,2016-04-21 08:44:49,8194,31,0.009,04-21,8
471142,0,2016-04-21 08:44:49,8194,529,0.002,04-21,8
345628,0,2016-04-21 08:44:49,8194,857,0.579,04-21,8
347034,0,2016-04-21 08:44:50,8194,31,0.307,04-21,8


time: 15 ms


In [8]:
user_0_state_set = user_0_data_first3days.groupby(["loc", "cat_id"]).size().reset_index().reset_index()
user_0_state_set.columns = ['state_code','loc_id','cat_id','count']
user_0_state_set

,state_code,loc_id,cat_id,count
0,0,1361,13,4
1,1,1361,15,13
2,2,1611,0,2
3,3,1611,13,5
4,4,1611,15,1
5,5,1611,4,1
6,6,1736,15,8
7,7,1736,4,2
8,8,1758,0,4
9,9,1758,13,3


time: 16 ms


In [9]:
user_0_state = pd.merge(user_0_data, user_0_state_set,  how='left', left_on=['loc','cat_id'], right_on = ['loc_id','cat_id'])
user_0_state = user_0_state.drop(columns= ["loc_id"])
user_0_state

,uid,datetime,loc,app_id,traffic,day,cat_id,state_code,count
0,0,2016-04-21 08:42:21,8194,342,0.030,04-21,0,152.000,13.000
1,0,2016-04-21 08:44:12,8194,1,0.008,04-21,4,156.000,4.000
2,0,2016-04-21 08:44:47,8194,857,0.027,04-21,8,158.000,59.000
3,0,2016-04-21 08:44:48,8194,857,0.002,04-21,8,158.000,59.000
4,0,2016-04-21 08:44:49,8194,612,0.001,04-21,8,158.000,59.000
5,0,2016-04-21 08:44:49,8194,4,0.002,04-21,13,153.000,41.000
6,0,2016-04-21 08:44:49,8194,31,0.009,04-21,8,158.000,59.000
7,0,2016-04-21 08:44:49,8194,529,0.002,04-21,8,158.000,59.000
8,0,2016-04-21 08:44:49,8194,857,0.579,04-21,8,158.000,59.000
9,0,2016-04-21 08:44:50,8194,31,0.307,04-21,8,158.000,59.000


time: 31 ms


In [10]:
user_0_train_stat = user_0_state[~(user_0_state['day'] == '04-26')]
user_0_test_stat = user_0_state[user_0_state['day'] == '04-26']
print(user_0_train_stat.shape, user_0_test_stat.shape)

(3248, 9) (663, 9)
time: 0 ns


In [58]:
state_train = list(np.int_(user_0_train_stat['state_code']))

time: 0 ns


In [60]:
def transition_matrix(transitions):
    #number of states
    n = max(transitions)
    M = [[0]*n for _ in range(n)]

    for (i,j) in zip(transitions,transitions[1:]):
        M[i-1][j-1] += 1

    #now convert to probabilities:
    for row in M:
        s = sum(row)
        if s > 0:
            row[:] = [f/s for f in row]
    return M

time: 0 ns


In [66]:
transition_matrix(state_train)

IndexError: list index out of range

In [16]:
actaul_state_code = np.array(user_0_state["state_code"])
predit_state_code = [np.argmax(user_0_transmat[i]) for i in actaul_state_code]

time: 0 ns


In [17]:
actaul_state_code = actaul_state_code[1:]
predit_state_code = predit_state_code[:-1]

time: 0 ns


In [18]:
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(actaul_state_code, predit_state_code)

0.24229585697328745

time: 563 ms


In [19]:
from sklearn.metrics import accuracy_score
accuracy_score(actaul_state_code, predit_state_code)

0.4048593350383632

time: 16 ms


In [20]:
from sklearn.metrics import f1_score
f1_score(actaul_state_code, predit_state_code, average='macro')

0.2049360834705059

time: 15 ms


In [21]:
from sklearn.metrics import confusion_matrix
confusion_matrix(actaul_state_code, predit_state_code)

array([[ 2,  0,  0, ...,  0,  0,  0],
       [ 2,  8,  0, ...,  0,  0,  0],
       [ 0,  0,  1, ...,  0,  0,  0],
       ...,
       [ 0,  0,  0, ..., 10,  2,  0],
       [ 0,  0,  0, ...,  0, 29,  0],
       [ 0,  0,  0, ...,  0,  0,  0]], dtype=int64)

time: 0 ns


# Transitional matrix for N random user

In [138]:
##Random sample

import random
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

def transition_matrix(transitions):
    #number of states
    n = 1+max(transitions)
    M = [[0]*n for _ in range(n)]

    for (i,j) in zip(transitions,transitions[1:]):
        M[i][j] += 1

    #now convert to probabilities:
    for row in M:
        s = sum(row)
        if s > 0:
            row[:] = [f/s for f in row]
    return M

def random_sample(n_users,n_times):
    all_b_acc = []
    all_acc = []
    all_f1 = []
    user_list = pd.unique(cleaned_df_cat['uid'])
    for i in range(n_times):
        random_50_user = random.choices(user_list, k = int(n_users))
        rand_50_user_df = cleaned_df_cat[cleaned_df_cat["uid"].isin(random_50_user)]
        state_50_rand_user = rand_50_user_df.groupby(["loc", "cat_id"]).size().reset_index().reset_index()
        state_50_rand_user.columns = ['state_code','loc_id','cat_id','count']
        users_state = pd.merge(rand_50_user_df, state_50_rand_user,  how='left', left_on=['loc','cat_id'], right_on = ['loc_id','cat_id'])
        users_state = users_state.drop(columns= ["loc_id"])
        users_state = users_state.sort_values(by = ['uid', 'datetime'])
        series = np.array(users_state["state_code"])
        trans_mat_all = transition_matrix(series)
        trans_mat_all = np.matrix(trans_mat_all)
        actaul_state_code = np.array(users_state["state_code"])
        predit_state_code = [np.argmax(trans_mat_all[i]) for i in actaul_state_code]
        actaul_state_code = actaul_state_code[1:]
        predit_state_code = predit_state_code[:-1]
        b_acc = balanced_accuracy_score(actaul_state_code, predit_state_code)
        acc = accuracy_score(actaul_state_code, predit_state_code)
        f1 = f1_score(actaul_state_code, predit_state_code, average='macro')
        all_b_acc.append(b_acc)
        all_acc.append(acc)
        all_f1.append(f1)
    print("Mean balanced Acc: {:.2f}, Mean Acc: {:.2f}, Mean f1-score: {:.2f} on random {} users for {} times".format(np.mean(all_b_acc),np.mean(all_acc), np.mean(all_f1), n_users, n_times))
    return

time: 0 ns


In [137]:
random_sample(50, 50)

Mean balanced Acc: 0.49, Mean Acc: 0.78, Mean f1-score: 0.47 on random 50 users for 50 times
time: 5min 21s


### Compute the entire transitonal matrix for all user will lead to out of memory problem, random sampling could be a solution